In [1]:
import os, sys
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

home_dir = os.environ['HOME']
sys.path.insert(0, os.path.join(home_dir, 'Dropbox/git/CMSC773'))
sys.path.insert(0, os.path.join(home_dir, 'Dropbox/git/bertviz'))
from bertviz import head_view

import dataloader
import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
import slda
import post_classifier
import bow

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('dark')

### Load Model

In [2]:
import torch
from deep_models.model import Bert4Clf
model = Bert4Clf(use_cuda=True, joint_training=True)

Total number of classifier parameters: 1538
Total number of BERT parameters: 109482240


In [3]:
model.load_state_dict(torch.load('./model_filtered_3epochs.bin'))

<All keys matched successfully>

### Load expert data

In [4]:
from deep_models.utils import load_data

file_name = 'expert_filtered_test_data.npy'
input_ids, attention_masks, labels, user_ids = load_data(file_name)

input_ids.shape, attention_masks.shape, labels.shape, user_ids.shape

((37519, 512), (37519, 512), (37519,), (37519,))

## viz

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [44]:
def show_head_view(model, inputs, attn_slice=range(30)):
   
    inputs_pt = list(map(lambda z: torch.tensor(
        z, dtype=torch.long, device=model.device).unsqueeze(0), inputs))

    with torch.no_grad():
        attention = model.bert(inputs_pt[0], attention_mask=inputs_pt[1])[-1]
        
    attention = (item * np.sum(inputs[1] > 0) / len(attn_slice) for item in attention)
        
    indices = list(attn_slice)
    if 0 not in indices:
        indices.append(0)
    indices = sorted(indices)
        
    attention = tuple(map(lambda z: z[..., indices][..., indices, :], attention))
    print(len(attention), attention[0].size())

    input_id_list = inputs[0][indices]
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    head_view(attention, tokens)

In [45]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [47]:
sample_i = 6624

inputs = input_ids[[sample_i]]

non_mask_inds = np.where(attention_masks[sample_i].flatten() > 0)[0]
non_masked_inputs = [inputs[0][non_mask_inds], attention_masks[sample_i][non_mask_inds]]

len(non_mask_inds), labels[sample_i]

(114, 1)

In [52]:
show_head_view(model, non_masked_inputs, attn_slice=range(57, 75))

12 torch.Size([1, 12, 19, 19])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>